In [1]:
import pandas as pd
import numpy as np
import seaborn as sea
import datetime as dt
import matplotlib.pyplot as pp
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

In [14]:
def split_seq(seq,input,output):
    X, Y = list(), list()
    for i in range(len(seq)):
        end_pat = i + input
        out_end = end_pat + output
        if out_end > len(seq):
            break
        seq_x, seq_y = seq[i:end_pat,:], seq[end_pat:out_end, :]
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X), np.array(Y)
dataset = pd.read_csv('in.csv', header = 0, index_col = 0)
in_seq1 = dataset['date_id']
in_seq2 = dataset['prev_cases']
out_seq = dataset['new_cases']
in_seq1 = in_seq1.values.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.values.reshape((len(in_seq2), 1))
out_seq = out_seq.values.reshape((len(out_seq), 1))
dataset = np.hstack((in_seq1,in_seq2,out_seq))
n_steps_in, n_steps_out = 3,2
X, Y = split_seq(dataset, n_steps_in, n_steps_out)
n_features = X.shape[2]
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.fit(X,Y,epochs=300,verbose=0)
x_input = dataset
x_input = np.arange(99621).reshape(3, 33207 )
x_input = x_input.reshape((x_input.shape[0]*6, 3, 3))
x_input = x_input.reshape((1, n_steps_in,n_features))
yhat = model.predict(x_input, verbose=0)

ValueError: cannot reshape array of size 99621 into shape (18,3,3)

In [8]:
dataset.shape

(33207, 3)

In [11]:
x_input.shape

(3, 33207)